<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class8-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 8.2**

**Announcements**
* Code for your Final Project will be due Monday, Nov. 21
* The perceptual tests will be released on Monday, Nov. 28 (right after Thanksgiving break)
  * Will take 1/2 hour to an hour
  * Due in 24 hours
* Final project writeup due at end of Finals Week 
  * Report template will be available next week

# **Final Project: Codec Evaluator**

In [ ]:
#@title Provided imports

import soundfile as sf
import IPython.display as ipd
import numpy as np
from scipy import signal
import time
import pickle

In [ ]:
#@title Mount Google Drive to access audio samples
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
load_path_1 = '/content/drive/MyDrive/eces435-work/Labs/Lab5/data/CoolSong_mono.wav'
load_path_2 = '/content/drive/MyDrive/eces435-work/Labs/Lab5/data/WildSong_mono.wav'
load_path_3 = '/content/drive/MyDrive/eces435-work/Labs/Lab5/data/ChillSong_mono.wav'
paths = [load_path_1,load_path_2,load_path_3]

## Load audio files into list `y`
* This loads the first 20 seconds only

In [ ]:
y = []
fs = []
dur = 20    # Load first 20 seconds

for p in paths:
  y_i, fs_i = sf.read(p)
  y_i = np.float16(y_i) #This will make sure that the audio you are comparing with for the project is 16 bit
  display(ipd.Audio(y_i[0:fs_i*dur], rate=fs_i))
  y.append(y_i[0:fs_i*dur])
  fs.append(fs_i)

## Your Codec System

*Your code goes here*

In [ ]:
#@title Your IMPORTS
'''
If you use any additional imports put them in this cell,
along with a comment about why you're using it
(what's its purpose?)
'''


In [ ]:
#@title Your ENCODER 

def encode(y, fs): 
  #Paste Encoder Code here. Inputs and Outputs MUST be the same as seen here.
  
  f1, t1, X = signal.stft(y, 44100, nperseg=1024, noverlap=512, nfft=1024)
  n_freqs, n_frames = np.shape(X)
  N_c = 60

  X_mag = np.abs(X)
  x_sortIdx = np.argsort(X_mag, axis=0)
  
  fft_idx = np.int16(x_sortIdx[-N_c:,:])
  fft_real = np.zeros([N_c, n_frames], dtype='float16')
  fft_imag = np.zeros([N_c, n_frames], dtype='float16')

  for n in range(n_frames):
    X_sort = X[fft_idx[:,n], n]
 
    fft_real[:,n] = np.real(X_sort)
    fft_imag[:,n] = np.imag(X_sort)

  y_compressed = [fft_idx, fft_real, fft_imag]
  return y_compressed              

In [ ]:
#@title Your DECODER

def decode(y_compressed):
  #Paste Decoder Code here. Inputs and Outputs MUST be the same as seen here.

  fft_idx = y_compressed[0]
  fft_real = y_compressed[1]
  fft_imag = y_compressed[2]

  n_freqs, n_frames = np.shape(fft_idx)
  X_c = 1j * np.zeros([513, n_frames])

  for n in range(n_frames):
    X_c[fft_idx[:,n],n] = fft_real[:,n] + 1j*fft_imag[:,n]

  t_c, y_reconstructed = signal.istft(X_c, 44100, nperseg=1024, noverlap=512, nfft=1024)

  return y_reconstructed

## Test Your Codec

In [ ]:
#@title Run Codec on Audio Samples
ert_ave = 0
drt_ave = 0

y_compressed = []
y_reconstructed = []
# S_reconstructed = []

for i in [0,1,2]:                       # Loops through the test audio samples
  sample = y[i]
  rate = fs[i]

  encodeStartTime = time.time()         # Starts the execution timer
  y_compressed_i = encode(sample,rate)  # Encode the audio sample
  ert = time.time() - encodeStartTime   # Saves the encoder execution time
  y_compressed.append(y_compressed_i)

  decodeStartTime = time.time()         # Start the execution timer
  y_reconstructed_i = decode(y_compressed_i)  # Decode the compressed audio
  drt = time.time() - decodeStartTime   # Saves the decoder execution time

  y_reconstructed.append(y_reconstructed_i)
  
  ert_ave += ert                        # Sum the encode exectution time
  drt_ave += drt                        # Sum the decode execution time

ert_ave /= 3                            # Average encode execution times
drt_ave /= 3                            # Average decode execution times
print(ert_ave)
print(drt_ave)

In [ ]:
#@title Compute Compression Ratio

def compression(song, song_compressed):
  Size_y = len(pickle.dumps(song))
  print('Size of original file: ', Size_y)
  Size_y_compressed = len(pickle.dumps(song_compressed))
  print('Size of compressed file: ', Size_y_compressed)
  Comp_Ratio = Size_y_compressed / Size_y
  print('Compression ratio: ', Comp_Ratio)
  
  return Comp_Ratio

comp_ave = 0

for i in [0,1,2]:
  Compression_Ratio = compression(y[i], y_compressed[i])
  comp_ave += Compression_Ratio

comp_ave /= 3
print("Average Compression Ratio: ", comp_ave)

In [ ]:
#@title Compute SNR

def SNR(original, output):
    # Normalize
    # original = original/np.max(np.abs(original)) # Normalize to -1 to 1
    # output =  output/np.max(np.abs(output)) # Normalize to -1 to 1
    
    # Find the shorter of the original vs. output
    L = min(len(original), len(output))

    # Compute SNR
    noise = original[:L] - output[:L]
    
    powS = np.mean(original**2)
    powN = np.mean(noise**2)

    snr = powS/powN
    snr = 10*np.log10(snr)
    return snr

snr_ave = 0

for i in [0,1,2]:
  snr = SNR(y[i],y_reconstructed[i])
  print("SNR for compressed audio: ", snr)

  snr_ave += snr

snr_ave /= 3
print("Average SNR: ", snr_ave)

## Codec Overall Scores

In [ ]:
#@title Display Overall Codec Results
print("Results Averaged over Three Test Samples:")
print("--------------------------------------------")
print("Avg. Compression Ratio")
print(comp_ave)
print()
print("Avg. SNR")
print(snr_ave)
print()
print("Avg. Run Time ")
print("Encode:", ert, "      Decode:", drt)
print()
print("Audio for Perception Test")
display(ipd.Audio(y_reconstructed[0], rate=fs[0]))
display(ipd.Audio(y_reconstructed[1], rate=fs[1]))
display(ipd.Audio(y_reconstructed[2], rate=fs[2]))

# Setup
The usual modules...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal
import numpy.fft as fft
import pickle
import matplotlib.patches as patches
import matplotlib.colors
import matplotlib.cm

rc('animation', html='jshtml')
rc('animation', embed_limit=2**30)

path = '/content/drive/My Drive/eces435-work/class8.1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'ub40'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Psychoacoustic Masking

In [ ]:
def ptcAnimFunc(f_num):
  # global out
  
  # print(f_num)
  freqs = np.arange(100,3000,250)
  f_c = freqs[f_num]

  fs = 44100

  A0 = 0.01
  f0 = 1000
  t = np.arange(fs) / fs
  N = len(t)
  win = np.hanning(N)
  s0 = A0 * np.sin(2*np.pi*f0*t) * win
  n_fft = N

  noise = np.random.uniform(-1, 1, N)
  f1 = max(f_c - 100, 1.0 ) / (fs/2)
  f2 = (f_c + 100) / (fs/2)
  B,A  = signal.butter(2, [f1,f2], btype='bandpass')
  n_f = signal.lfilter(B,A,noise) * win
  x_n = s0 + n_f
  # out = np.append(out, x_n)

  X_n = np.fft.fft(x_n * win, n_fft)  # The FFT of the current frame (windowed)
  N = len(X_n)
  f = np.arange(N) * fs / N
  X_mag = 4*np.abs(X_n) / n_fft   # Frequency magnitude, normalized by length
                                  #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                  #    x2 because hanning window has 0.5 average
  X_mag += 1.0e-15                # Add a very small offset to avoid log(0) errors
  X_dBFS = 20*np.log10(X_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w
  fftLine.set_data(f, X_dBFS)
  return fftLine,

In [ ]:
def ptc():
  fs = 44100
  A0 = 0.01
  f0 = 1000
  t = np.arange(fs) / fs
  N = len(t)
  win = np.hanning(N)
  s0 = A0 * np.sin(2*np.pi*f0*t) * win
  noise = np.random.uniform(-1, 1, N)
  out = np.zeros(1)
  
  for f_c in np.arange(100,3000,250):  
    f1 = max(f_c - 100, 1.0 ) / (fs/2)
    f2 = (f_c + 100) / (fs/2)
    B,A  = signal.butter(2, [f1,f2], btype='bandpass')
    n_f = signal.lfilter(B,A,noise) * win
    x_n = s0 + n_f
    out = np.append(out, x_n)

  return out

In [ ]:
# Set up the figure, the axis, and the plot element we want to animate
x_lim=[0, 22050]
y_lim=[-120,5]
out = np.zeros(1)
fs = 44100

fftFig = plt.figure()
ax = plt.axes(xlim=x_lim, ylim=y_lim)
fftLine, = ax.plot([], [], linewidth=2)

plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB FS)')
fftFig.tight_layout()
plt.close()             # Close the plot, so we don't get an extra figure added on

# Call the animation iterator (FuncAnimation)
animation.FuncAnimation(fftFig, ptcAnimFunc, frames=12, blit=True)

In [ ]:
out = ptc()
ipd.Audio(out, rate=fs)